In [16]:
import sys
import numpy as np
from random import randint
import random
import math
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import operator
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as GNB
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from deap import creator, base, tools, algorithms

In [24]:
file1 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/commenting_data/train.csv'
df = pd.read_csv(file1)
print(df.head())

corpus = list(df["Description"])
target = list(df["Category"])


                                         Description  Category
0  Was walking along crowded street, holding mums...         0
1  This incident took place in the evening.I was ...         0
2  I WAS WAITING FOR THE BUS. A MAN CAME ON A BIK...         1
3                 Incident happened inside the train         0
4  I witnessed an incident when a chain was bruta...         0


In [25]:
infinity = float("inf") #sentinel maximum value
m = 70 #no of bats (50-70)
T = 100 # no of iterations(100)

bat_pos = defaultdict(list)
bat_vel = defaultdict(list)
bat_loudness = []
bat_rate = []
bat_fitness = []
bat_frequency = []

global_bat_pos = []
init_rate_vector = []
feature_vector = []

#hyper parameters
alpha = 0.9
gamma = 0.9
epsilon = random.uniform(-1,1)
fmin=0
fmax =1

In [26]:
#do once
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(ngram_range=(1, 3), stop_words=stop_words)
X = vectorizer.fit_transform(corpus)
print(X.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))

X_train = X
Y_train = target

file2 = '/Users/sukrverm/Documents/NLP-Review/safecity/binary_classification/commenting_data/test.csv'
df_test = pd.read_csv(file2)
print(df_test.head())
corpus_test = list(df_test["Description"])
target_test = list(df_test["Category"])
X_test = vectorizer.transform(corpus_test)
print(X_test.shape)
print(vectorizer.get_feature_names()[0:50])
print(len(vectorizer.get_feature_names()))
Y_test = target_test


(7201, 121356)
[u'00', u'00 confronted', u'00 confronted hit', u'00 go', u'00 go downstairs', u'00 way', u'00 way met', u'000', u'000 frs', u'000 frs could', u'000 frs parifoot', u'000 rupees', u'000 rupees man', u'00am', u'00am morning', u'00am morning tried', u'00pm', u'01', u'01 2011', u'01 2011 walking', u'03', u'03 2013', u'03 2013 scooty', u'03 29', u'03 29 2013', u'03 aa', u'03 aa 5562', u'0300', u'0300 pm', u'0300 pm time', u'05', u'05 2013', u'05 2013 evening', u'05 t6644', u'05 t6644 bus', u'06', u'06 2013', u'0740', u'0740 pm', u'0740 pm persons', u'0800am', u'0800am saw', u'0800am saw watchman', u'0830', u'0830 0900', u'0830 0900 sunday', u'08826862360', u'0900', u'0900 sunday', u'0900 sunday street']
121356
                                         Description  Category
0  During morning, a woman was walking by and thi...         1
1  A man tried to brush his penis off of a woman'...         0
2  This happened to a fellow passenger of mine tr...         0
3                 

In [27]:
print(X_train.shape)
z1 = (X_train)
z2 = (X_test)
print(z1.shape, z2.shape)
n = z1.shape[1]

(7201, 121356)
((7201, 121356), (1701, 121356))


In [28]:
#auxillary  functions
def extract_subset(z,cols):
    return z[:,cols]

In [29]:
def svm_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(X_train, cols)
    z2_prime = extract_subset(X_test, cols)
    clf = svm.LinearSVC(C=1.0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall



In [ ]:
def rf_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = RandomForestClassifier(n_jobs=2,random_state=0)
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def knn_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = KNN(3,weights='uniform')
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def nb_calc_fitness(x):
    for i in range(0,len(x)):
        x[i] = x[i] > 0.5
    cols = []
    for j in range(0, n):
        if x[j] == 1:
            cols.append(j)
    z1_prime = extract_subset(z1, cols)
    z2_prime = extract_subset(z2, cols)
    clf = GNB()
    clf.fit(z1_prime, Y_train)
    y_pred = clf.predict(z2_prime)
    acc = sklearn.metrics.accuracy_score(Y_test,y_pred)
    pre = sklearn.metrics.precision_score(Y_test,y_pred)
    fscore = sklearn.metrics.f1_score(Y_test,y_pred)
    recall = sklearn.metrics.recall_score(Y_test,y_pred)
    return (acc),pre,fscore,recall

In [ ]:
def initialise(m,n):
    for i in range(0,m):
        for j in range(0,n):
            x = randint(0,1)
            bat_pos[i].append(x)
            bat_vel[i].append(0)

        init_rate_vector.append(random.uniform(0, 1))

        bat_frequency.append(init_rate_vector[i])
        bat_loudness.append(random.uniform(1,2))
        bat_rate.append(init_rate_vector[i])
        bat_fitness.append(-infinity)

In [ ]:
def core():
    initialise(m,n)
    global_fit = -infinity
    for t in range (0,T):
        print("T:" , t)
        for i in range(0,m):
            accuracy,pre,fscore,recall = svm_calc_fitness(bat_pos[i])
            '''
            with open("dumpPerformanceBat_svm_kpi.txt", "a") as text_file:
                text_file.write("{} {} {} {} {}".format(accuracy,pre,fscore,recall,global_fit) + "\n")
            with open("dumpPerformanceBat_svm.txt", "a") as text_file:
                text_file.write("{} {} {} {}".format(t,accuracy,bat_pos[i].count(1),global_fit)+"\n")
            #print("t={} accuracy={} #features={} global_fit={}".format(t,accuracy,bat_pos[i].count(1),global_fit))
            '''
            acc = accuracy

            rand = randint(0,1)
            if rand < bat_loudness[i] and acc > bat_fitness[i]:
                bat_fitness[i]=acc
                bat_loudness[i]= alpha * bat_loudness[i]
                bat_rate[i]= init_rate_vector[i] * (1 - math.exp(-(gamma * t)))

        maxindex,maxfit = max(enumerate(bat_fitness), key=operator.itemgetter(1))

        if maxfit>global_fit :
            global_fit = maxfit
            global_bat_pos = bat_pos[maxindex]

        for i in range(0,m): # for each bat
            beta = random.uniform(0,1)
            rand = random.uniform(0,1)
            AvgA = sum(bat_loudness)/len(bat_loudness)

            if rand > bat_rate[i] :
                for j in range(0,n):
                    bat_pos[i][j] = bat_pos[i][j] + ( epsilon * AvgA )
                    sigma = randint(0,1)
                    if sigma < (1/(1+math.exp(-bat_pos[i][j]))) :
                        bat_pos[i][j]=1
                    else :
                        bat_pos[i][j]= 0

            rand = random.uniform(0,1)
            if rand < bat_loudness[i] and bat_fitness[i] < global_fit :
                for j in range (0,n):
                    bat_frequency[i] = fmin + (fmax-fmin)* beta
                    bat_vel[i][j] = bat_vel[i][j] + (global_bat_pos[j]-bat_pos[i][j]) * bat_frequency[i]
                    bat_pos[i][j] = bat_pos[i][j] + bat_vel[i][j]
                    sigma = random.uniform(0,1)
                    if sigma < (1 / (1 + math.exp(-bat_pos[i][j]))) :
                        bat_pos[i][j]=1
                    else :
                        bat_pos[i][j]=0
        with open("dumpPerformanceBat_rf.txt", "a") as text_file:
            text_file.write("{} {} {} ".format(t,global_bat_pos.count(1),global_fit)+"\n")
        


    feature_vector = global_bat_pos
    return feature_vector

In [ ]:
def BAT_algo():
    feature_vector = core()
    return feature_vector

In [ ]:
f = BAT_algo()
 

In [30]:
class GeneticAlgorithm():
    
    def __init__(self, X_tr, X_te, Y_train, Y_test, number_bits_to_mutate = 25.0):
        
        
        self.toolbox = base.Toolbox()
        self.ind_mut_pb = float(number_bits_to_mutate / X_tr.shape[1])
        self.NGEN = 100
        self.MUTPB = 0.2
        self.CXPB = 0.5
        self.POPULATION_SIZE = 70
        self.NUMBER_TO_SELECT_FROM_POP = 35
        self.init_pop = list()
        self.X_tr = X_train
        self.X_te = X_test
        self.Y_train = Y_train
        self.Y_test = Y_test
        
    
    def initialise_toolbox(self):
        print('now initializing toolbox GA1')
        # I want to maximize session overlap as well as length
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        # representing individual as a list
        creator.create("Individual", list, fitness=creator.FitnessMax)
        # individual will be a list of 0/1
        self.toolbox.register("attr_bool", random.randint, 0, 1)
        IND_DIMENSIONALITY = self.X_tr.shape[1]
        # create template for individual
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_bool, n=IND_DIMENSIONALITY)
        #self.toolbox.register("individual", tools.initIterate, creator.Individual, self.get_ind)
        # create template for population
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        print('toolbox initialised')
    
    
    def initialise_fitness_function(self):
        print('building and registering the fitness function')
        
        def fitness(individual):
            
            
            accuracy,pre,fscore,recall = svm_calc_fitness(individual)
            return fscore,
            
        
        self.toolbox.register("evaluate", lambda individual: fitness(individual))
        self.toolbox.register("mate", tools.cxOnePoint)
        self.toolbox.register("mutate", tools.mutFlipBit, indpb=self.MUTPB)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
        print('fitness function ready')
    
    def run_algorithm(self):
        
        self.initialise_toolbox()
        self.initialise_fitness_function()
        
        print('mutation probability was calculated at %f' % self.ind_mut_pb)
        print('running the algorithm')
        toolbox = self.toolbox
        # create a population of size n
        pop = toolbox.population(n=self.POPULATION_SIZE)
        
        
        for gen in range(self.NGEN):
            # Select the next generation individuals
            if gen%5 == 0:
                print(gen)
            offspring = toolbox.select(pop, len(pop))
            
            for ind in offspring:
                if ind.fitness.valid:
                    pass
            # Clone the selected individuals
            offspring = map(toolbox.clone, offspring)

            # Apply crossover on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.CXPB:
                    #toolbox.mate(child1, child2)
                    tools.cxUniform(child1, child2, 0.5)
                    del child1.fitness.values
                    del child2.fitness.values

            # Apply mutation on the offspring
            for mutant in offspring:
                if random.random() < self.MUTPB:
                    # print('now doing mutation')
                    toolbox.mutate(mutant)
                    del mutant.fitness.values

            # Evaluate the individuals with an invalid fitness
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            
            added = 0
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
                

            # The population is entirely replaced by the offspring
            pop[:] = offspring
            top = tools.selBest(pop, k=self.NUMBER_TO_SELECT_FROM_POP)
            
            # Dump best individual to file
            fittest = tools.selBest(pop, k=1)[0]
            count = 0
            for i in range(0, len(fittest)):
                if fittest[i] == 1:
                    count = count + 1 
            with open("GA_performance.txt", "a") as text_file:
                accuracy,prec,fscore,recall = svm_calc_fitness(fittest)
                text_file.write("{} {}".format(count,fittest.fitness.values[0])+"\n")
                print(count,accuracy,prec,fscore,recall,fittest.fitness.values[0])
        
        
        
        
        max_fitness = float("-inf")
        
        for individual in tools.selBest(pop, k=1):
            if individual.fitness.valid:
                if individual.fitness.values[0] > max_fitness:
                    max_fitness = individual.fitness.values[0]
                    fittest_individual = individual
                    
        count = 0
        for i in range(0, len(fittest_individual)):
            if fittest_individual[i] == 1:
                count = count + 1        
        
        print(count, max_fitness)

In [31]:
ga = GeneticAlgorithm(X_train, X_test, Y_train, Y_test)

In [32]:
ga.run_algorithm()

now initializing toolbox GA1
toolbox initialised
building and registering the fitness function
fitness function ready
mutation probability was calculated at 0.000206
running the algorithm
0
(60373, 0.8124632569077014, 0.8096118299445472, 0.7330543933054393, 0.6697247706422018, 0.7330543933054393)
(60373, 0.8124632569077014, 0.8096118299445472, 0.7330543933054393, 0.6697247706422018, 0.7330543933054393)
(60373, 0.8124632569077014, 0.8096118299445472, 0.7330543933054393, 0.6697247706422018, 0.7330543933054393)
(60616, 0.8130511463844797, 0.798932384341637, 0.7384868421052632, 0.6865443425076453, 0.7384868421052632)
(60377, 0.8177542621987066, 0.8028169014084507, 0.7463175122749591, 0.6972477064220184, 0.7463175122749591)
5
(60377, 0.8177542621987066, 0.8028169014084507, 0.7463175122749591, 0.6972477064220184, 0.7463175122749591)
(60377, 0.8177542621987066, 0.8028169014084507, 0.7463175122749591, 0.6972477064220184, 0.7463175122749591)
(60377, 0.8177542621987066, 0.8028169014084507, 0.746

(60545, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60556, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60575, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60563, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60570, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
80
(60583, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60542, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60590, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60567, 0.8330393885949442, 0.8257042253521126, 0.7675941080196399, 0.7171253822629969, 0.7675941080196399)
(60552, 0.833039388594944